# Custom map simulations

In this tutorial we will build a simulation from scratch.

We start by defining a `Band` that will determine our array's sensitivity to different spectra. We then generate an array by specifying a field of view, which will be populated by evenly-spaced beams of the given band.

In [ ]:
import maria
from maria.instrument import Band

f090 = Band(
    center=90e9,  # in Hz
    width=20e9,  # in Hz
    NET_RJ=40e-6,  # in K sqrt(s)
    knee=1e0,    # in Hz
    gain_error=5e-2)

f150 = Band(
    center=150e9, 
    width=30e9, 
    NET_RJ=60e-6, 
    knee=1e0, 
    gain_error=5e-2)

In [ ]:
array = {"field_of_view": 0.1, 
         "shape": "circle", 
         "beam_spacing": 1.5,
         "primary_size": 100, 
         "bands": [f090, f150]}

instrument = maria.get_instrument(array=array)

print(instrument)
instrument.plot()

As something to observe, we can download a map and construct a `map`. We also define a plan to do a daisy scan centered on the center of the map.

In [ ]:
from maria.io import fetch

map_filename = fetch("maps/tarantula_nebula.h5")

input_map = maria.map.load(
    filename=map_filename,
    nu=150e9,
    width=0.25,
    center=(291.156, -31.23),
    units="uJy/pixel")

print(input_map)
input_map.to("K_RJ").plot()

In [ ]:
site = maria.get_site("llano_de_chajnantor", altitude=5065)

print(site)
site.plot()

In [ ]:
plan = maria.Plan(
    start_time="2024-08-06T03:00:00",
    scan_pattern="daisy",
    scan_options={"radius": 0.08, "speed": 0.01},  # in degrees
    duration=600,  # in seconds
    sample_rate=50,  # in Hz
    scan_center=(291.156, -31.23),
    frame="ra_dec")

print(plan)
plan.plot()

In [ ]:
sim = maria.Simulation(
    instrument,
    plan=plan,
    site=site,
    atmosphere="2d",
    atmosphere_kwargs={"weather": {"pwv": 0.5}},
    cmb="generate",
    map=input_map)

print(sim)

In [ ]:
tod = sim.run()

print(tod)
tod.plot()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from maria.functions.radiometry import inverse_rayleigh_jeans_spectrum, planck_spectrum

band = f090

T_b = np.array([2.7, 3.1])
# T_b = 2.8

spectrum = sim.atmosphere.spectrum

test_T_RJ = inverse_rayleigh_jeans_spectrum(
        planck_spectrum(T_b=np.atleast_2d(T_b), nu=spectrum.side_nu[:, None]),
        nu=spectrum.side_nu[:, None],
    )

integral = np.trapezoid(
    y=test_T_RJ[None, None, None] * (np.exp(-spectrum._opacity) * band.passband(spectrum.side_nu))[..., None],
    x=spectrum.side_nu,
    axis=-2,
)
# points = (test_T_b, *spectrum.points[:3])
# xi = (
#     T_b,
#     kwargs["base_temperature"],
#     kwargs["zenith_pwv"],
#     kwargs["elevation"],
# )
# #return k_B * sp.interpolate.interpn(points, integral, xi)

In [ ]:
f090

In [ ]:
integral.shape

In [ ]:
test_T_RJ.shape

In [ ]:
spectrum.side_nu[:, None].shape

In [ ]:
np.atleast_2d(T_b).shape

In [ ]:
planck_spectrum

In [ ]:
plt.plot(spectrum.side_nu, test_T_RJ)
plt.xscale("log")

In [ ]:

if spectrum:
    test_T_RJ = inverse_rayleigh_jeans_spectrum(
        planck_spectrum(T_b=T_b, nu=spectrum.side_nu),
        nu=spectrum.side_nu,
    )
    integral = np.trapezoid(
        y=test_T_RJ[:, None, None, None] * np.exp(-spectrum._opacity) * band.passband(spectrum.side_nu),
        x=spectrum.side_nu,
        axis=-1,
    )
    points = (test_T_b, *spectrum.points[:3])
    xi = (
        T_b,
        kwargs["base_temperature"],
        kwargs["zenith_pwv"],
        kwargs["elevation"],
    )
    return k_B * sp.interpolate.interpn(points, integral, xi)

else:
    test_T_RJ = inverse_rayleigh_jeans_spectrum(planck_spectrum(T_b=test_T_b[:, None], nu=band.nu), nu=band.nu)
    integral = np.trapezoid(y=test_T_RJ * band.passband(band.nu), x=band.nu, axis=-1)
    return k_B * sp.interpolate.interp1d(test_T_b, integral)(T_b)


In [ ]:
import numpy as np
import scipy as sp

self = sim

o = sp.interpolate.interp1d(self.atmosphere.spectrum.side_nu, 
                        self.atmosphere.spectrum._opacity)(f090.nu)

In [ ]:
np.ones_like(o).shape

In [ ]:
plt.plot(sim.atmosphere.spectrum.opacity(nu=f090.nu))

In [ ]:
spectrum

In [ ]:
%debug

In [ ]:
from maria.mappers import BinMapper

mapper = BinMapper(
    center=(291.156, -31.23),
    frame="ra_dec",
    width=0.25,
    height=0.25,
    resolution=0.25 / 256,
    tod_preprocessing={
        "window": {"name": "tukey", "kwargs": {"alpha": 0.1}},
        "remove_spline": {"knot_spacing": 30, "remove_el_gradient": True},
        "remove_modes": {"modes_to_remove": [0]},
    },
    map_postprocessing={
        "gaussian_filter": {"sigma": 1},
        "median_filter": {"size": 1},
    },
    units="mK_RJ",
)

mapper.add_tods(tod)

output_map = mapper.run()

We can see the recovered map with

In [ ]:
print(output_map)
output_map.plot()